In [1]:
import pandas as pd
import hvplot.pandas

%opts magic unavailable (pyparsing cannot be imported)
%compositor magic unavailable (pyparsing cannot be imported)


In [20]:
QUOTA = pd.DataFrame(data={"NodeHours":[8000]*12}, index=pd.date_range("2025-02-14",freq="MS",periods=12))
QUOTA["CumNodeHours"] = QUOTA["NodeHours"].cumsum()
PLAN = pd.DataFrame(data={"NodeHours":[3000]*3+[10000]*9}, index=pd.date_range("2025-02-14",freq="MS",periods=12))
PLAN["CumNodeHours"] = PLAN["NodeHours"].cumsum()

In [21]:
brey = pd.read_csv("sacct_energy_nodehours_brey.txt", sep="\t")
panos = pd.read_csv("sacct_energy_nodehours_panos.txt", sep="\t")
tomsail = pd.read_csv("sacct_energy_nodehours_tomsail.txt", sep="\t")

def add_info(df:pd.DataFrame, name):
    df["Submit"] = pd.to_datetime(df["Submit"], errors="coerce")
    df.index = df["Submit"]
    df = df[df["NodeHours"]>0]
    df =df.rename(columns={
        "NodeHours":f"{name} usage",
        })
    return df
brey = add_info(brey,"u101116")
tomsail = add_info(tomsail,"u101170")
panos = add_info(panos,"u101348")

In [22]:
usage = pd.concat([brey, tomsail, panos],axis=0).fillna(0)
usage = usage.sort_index()

bar = usage[["u101170 usage","u101116 usage","u101348 usage"]].resample("MS").sum().hvplot.bar(stacked=True)
(QUOTA["NodeHours"].hvplot.bar(c="grey",alpha=0.5, label="quota") * bar).opts(width=900, title ="Monthly usage, compared to quota allocated")
(PLAN["NodeHours"].hvplot.bar(c="grey",alpha=0.5, label="flexible quota") * bar).opts(width=900, title ="Monthly usage, compared to requested quota in proposal")

:Overlay
   .Bars.Quota :Bars   [index]   (NodeHours)
   .Bars.I     :Bars   [Submit,Variable]   (value)

:Overlay
   .Bars.Flexible_quota :Bars   [index]   (NodeHours)
   .Bars.I              :Bars   [Submit,Variable]   (value)

In [23]:
usage[["u101116 cumulated","u101348 cumulated", "u101170 cumulated"]] = usage[["u101116 usage","u101348 usage", "u101170 usage"]].cumsum()

In [24]:
usage = usage[~usage.index.duplicated(keep="first")]

In [25]:
QUOTA = QUOTA.rename(columns={"CumNodeHours":"Allocated Quota"})
PLAN = PLAN.rename(columns={"CumNodeHours":"Requested Quota"})
(usage[["u101116 cumulated","u101170 cumulated","u101348 cumulated", ]].hvplot.area() * 
 QUOTA["Allocated Quota"].hvplot.line(c='k', line_dash = "dashed") * 
 PLAN["Requested Quota"].hvplot.line(c='grey', line_dash = "dashed")).opts(
    xlim=(None, pd.Timestamp.now()),
    ylim=(0,50000),
    width=1000,
    height=600,
    )

:Overlay
   .NdOverlay.I           :NdOverlay   [Variable]
      :Area   [Submit]   (value,Baseline)
   .Curve.Allocated_Quota :Curve   [index]   (Allocated Quota)
   .Curve.Requested_Quota :Curve   [index]   (Requested Quota)

In [26]:
energy = (brey.ConsumedEnergyRaw.sum() + tomsail.ConsumedEnergyRaw.sum() + panos.ConsumedEnergyRaw.sum()) / 3.6e6
f"energy consumed: {energy:.2f} kWh"
co2 = energy * 409.8438/1000
f"corresponding to {co2:.2f} CO2 kg Emissions"

'energy consumed: 11772.67 kWh'

'corresponding to 4824.96 CO2 kg Emissions'

# Scaling

In [10]:
scale = pd.read_csv("scaling_hydrogen.csv", sep = "\s+|\t+|\s+\t+|\t+\s+")
scale_lumi = pd.read_csv("scaling_hydrogen_lumi.csv", sep = "\s+|\t+|\s+\t+|\t+\s+")
scale_leonardo = pd.read_csv("scaling_hydrogen_leonardo.csv", sep = "\s+|\t+|\s+\t+|\t+\s+")


def add_params(scale):
    scale["walltime"] = scale["steps_total"]
    scale["Tf*"] = scale["walltime"] * 60*60 * scale["no_tasks"] / 3e6
    scale["Speedup"] = scale["walltime"].iloc[-1] / scale["walltime"]
    scale["Ideal"] = scale["no_tasks"] / scale["no_tasks"].iloc[-1]
    scale["Efficiency"] = scale["Speedup"] / scale["Ideal"] * 100
    scale[["no_tasks","walltime","Tf*","Speedup","Ideal","Efficiency"]].iloc[::-1]
    return scale

scale_leonardo = add_params(scale_leonardo)
scale_lumi = add_params(scale_lumi)
scale = add_params(scale)

/tmp/ipykernel_1248406/2531848566.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  scale = pd.read_csv("scaling_hydrogen.csv", sep = "\s+|\t+|\s+\t+|\t+\s+")
/tmp/ipykernel_1248406/2531848566.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  scale_lumi = pd.read_csv("scaling_hydrogen_lumi.csv", sep = "\s+|\t+|\s+\t+|\t+\s+")
/tmp/ipykernel_1248406/2531848566.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.

In [19]:
(scale.hvplot.line(label = "meluxina", x="no_tasks", y="walltime", grid=True,logx=True, logy=True, ylabel="Walltime (hours per model day)",xlabel="Number of cores")*\
scale.hvplot.scatter(label = "meluxina", x="no_tasks", y="walltime", marker="o")*\
scale_lumi.hvplot.line(label = "lumi", x="no_tasks", y="walltime", grid=True,logx=True, logy=True, ylabel="Walltime (hours per model day)",xlabel="Number of cores")*\
scale_lumi.hvplot.scatter(label = "lumi", x="no_tasks", y="walltime", marker="o")*\
scale_leonardo.hvplot.line(label = "leonardo", x="no_tasks", y="walltime", grid=True,logx=True, logy=True, ylabel="Walltime (hours per model day)",xlabel="Number of cores")*\
scale_leonardo.hvplot.scatter(label = "leonardo", x="no_tasks", y="walltime", marker="o")
).opts(height=500, title = "Walltime (hours)")

:Overlay
   .Curve.Meluxina   :Curve   [no_tasks]   (walltime)
   .Scatter.Meluxina :Scatter   [no_tasks]   (walltime)
   .Curve.Lumi       :Curve   [no_tasks]   (walltime)
   .Scatter.Lumi     :Scatter   [no_tasks]   (walltime)
   .Curve.Leonardo   :Curve   [no_tasks]   (walltime)
   .Scatter.Leonardo :Scatter   [no_tasks]   (walltime)

In [18]:
(scale.hvplot.line(label="meluxina", x="no_tasks", y="Speedup", grid=True, ylabel="Speedup",xlabel="Number of cores")*\
scale.hvplot.scatter(label="meluxina",x="no_tasks", y="Speedup")*\
scale_lumi.hvplot.line(label="lumi", x="no_tasks", y="Speedup", grid=True, ylabel="Speedup",xlabel="Number of cores")*\
scale_lumi.hvplot.scatter(label="lumi", x="no_tasks", y="Speedup")*\
scale_leonardo.hvplot.line(label="leonardo", x="no_tasks", y="Speedup", grid=True, ylabel="Speedup",xlabel="Number of cores")*\
scale_leonardo.hvplot.scatter(label="leonardo", x="no_tasks", y="Speedup")*\
 scale.hvplot.line(x="no_tasks", y="Ideal", grid=True, label="ideal", c='k', line_dash="dashed")
 ).opts(height=500, legend_position = "bottom_right", title="Speedup graph")

:Overlay
   .Curve.Meluxina   :Curve   [no_tasks]   (Speedup)
   .Scatter.Meluxina :Scatter   [no_tasks]   (Speedup)
   .Curve.Lumi       :Curve   [no_tasks]   (Speedup)
   .Scatter.Lumi     :Scatter   [no_tasks]   (Speedup)
   .Curve.Leonardo   :Curve   [no_tasks]   (Speedup)
   .Scatter.Leonardo :Scatter   [no_tasks]   (Speedup)
   .Curve.Ideal      :Curve   [no_tasks]   (Ideal)

In [9]:
(scale.hvplot.line(x="no_tasks", y="Efficiency", grid=True, ylabel="Parallel efficiency (%)",xlabel="Number of cores")*\
 scale.hvplot.scatter(x="no_tasks", y="Efficiency")).opts(height=500, legend_position = "bottom_right")

:Overlay
   .Curve.I   :Curve   [no_tasks]   (Efficiency)
   .Scatter.I :Scatter   [no_tasks]   (Efficiency)

table for word document:

In [ ]:
scale